In [2]:
#Libraries
require(data.table, quietly = TRUE)
require(glue, quietly = TRUE)
require(ggplot2, quietly = TRUE)
require(scatterplot3d, quietly = TRUE)
require(gridExtra, quietly = TRUE)
require(tidyr, quietly = TRUE)
require(tseries, quietly = TRUE)
library(pROC, quietly = TRUE)
library(caret, quietly = TRUE)
library(rpart, quietly = TRUE)
library(rpart.plot, quietly = TRUE)
library(e1071, quietly = TRUE)

In [3]:
trainpath <- "Data/IE582_Fall20_ProjectTrain.csv"
traindata <- read.csv(trainpath)

submitdatapath <- "Data/IE582_Fall20_ProjectTest.csv"
submitdata <- read.csv(submitdatapath)

submitdata <- submitdata[,-61] # Empty y column of the submission data set

In [4]:
summary(traindata)

       x1              x2               x3              x4        
 Min.   :13.00   Min.   :0.0000   Min.   :0.000   Min.   :0.0000  
 1st Qu.:27.00   1st Qu.:0.0000   1st Qu.:0.000   1st Qu.:0.0000  
 Median :30.00   Median :1.0000   Median :1.000   Median :1.0000  
 Mean   :30.09   Mean   :0.6712   Mean   :0.662   Mean   :0.6905  
 3rd Qu.:33.00   3rd Qu.:1.0000   3rd Qu.:1.000   3rd Qu.:1.0000  
 Max.   :50.00   Max.   :1.0000   Max.   :1.000   Max.   :1.0000  
       x5               x6              x7               x8       
 Min.   : 0.000   Min.   : 0.00   Min.   : 0.000   Min.   :13.00  
 1st Qu.: 4.000   1st Qu.: 4.00   1st Qu.: 4.000   1st Qu.:26.00  
 Median : 9.000   Median : 9.00   Median : 9.000   Median :30.00  
 Mean   : 9.083   Mean   : 8.99   Mean   : 9.109   Mean   :30.17  
 3rd Qu.:14.000   3rd Qu.:14.00   3rd Qu.:14.000   3rd Qu.:34.00  
 Max.   :18.000   Max.   :18.00   Max.   :18.000   Max.   :49.00  
       x9              x10              x11              x12  

In [14]:
str(traindata)

'data.frame':	2074 obs. of  61 variables:
 $ x1 : int  27 30 37 29 33 33 29 27 28 27 ...
 $ x2 : int  1 0 0 0 1 0 1 1 1 0 ...
 $ x3 : int  1 1 1 1 1 0 0 1 1 1 ...
 $ x4 : int  1 1 1 1 0 1 1 1 1 0 ...
 $ x5 : int  18 18 1 14 2 5 16 13 0 8 ...
 $ x6 : int  3 13 3 9 15 5 1 4 0 18 ...
 $ x7 : int  1 3 14 3 12 12 2 17 2 18 ...
 $ x8 : int  28 19 33 29 39 26 24 34 40 26 ...
 $ x9 : num  119.9 86.7 174 8.8 55 ...
 $ x10: num  154 133 128 127 188 ...
 $ x11: num  121.4 129 100.2 55.5 156.6 ...
 $ x12: int  1 0 0 1 1 0 0 0 0 1 ...
 $ x13: int  0 0 0 0 0 0 0 0 0 0 ...
 $ x14: int  404 303 454 383 404 404 404 30 202 404 ...
 $ x15: int  1 1 1 1 0 1 0 1 1 1 ...
 $ x16: int  0 0 0 1 0 0 0 0 1 0 ...
 $ x17: int  0 0 0 0 0 0 0 0 0 0 ...
 $ x18: int  0 0 0 0 0 0 0 0 0 0 ...
 $ x19: int  0 0 0 0 0 0 0 0 0 0 ...
 $ x20: int  0 0 0 0 0 0 0 0 0 0 ...
 $ x21: int  0 0 0 0 0 0 0 0 0 0 ...
 $ x22: int  0 1 0 0 0 0 0 0 0 0 ...
 $ x23: int  1 1 1 0 1 0 1 0 0 0 ...
 $ x24: int  0 0 0 1 0 0 0 1 1 0 ...
 $ x25: i

In [36]:
factor2 <- c()
for(i in 1:60)
{
  if(nlevels(as.factor(traindata[,i]))==2)
  {
    factor2 <- c(factor2,i)
  }
}

In [40]:
sapply(traindata[,factor2],table)

,x2,x3,x4,x12,x13,x15,x16,x17,x18,x19,...,x49,x51,x53,x54,x55,x56,x57,x58,x59,x60
0,682,701,642,1363,2005,312,1840,1579,2032,1982,...,2053,1856,1895,1410,2038,1184,2072,1779,2061,2009
1,1392,1373,1432,711,69,1762,234,495,42,92,...,21,218,179,664,36,890,2,295,13,65


In [41]:
train2 = traindata[,factor2]

In [48]:
train3 = t(sapply(traindata[,factor2],table))

In [71]:
cdeneme <- c()
for(i in 1:nrow(train3))
{
  if(train3[i,2]<100)
  {
    cdeneme <- c(cdeneme, i)
  }
}

In [75]:
cdeneme2 <- c()
for(i in 1:nrow(train3))
{
  if(train3[i,1]<100)
  {
    cdeneme2 <- c(cdeneme2, i)
  }
}

In [77]:
train3[cdeneme,]

,0,1
x13,2005,69
x18,2032,42
x19,1982,92
x20,1979,95
x21,2015,59
x22,1976,98
x26,2057,17
x29,2003,71
x31,2004,70
x33,2005,69


In [80]:
train5 = traindata[,-cdeneme]

In [81]:
train5

x1,x2,x3,x4,x6,x7,x8,x14,x15,x16,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,y
27,1,1,1,3,1,28,404,1,0,...,0,0,0,0,1,0,0,0,0,a
30,0,1,1,13,3,19,303,1,0,...,0,0,0,0,1,0,0,0,0,a
37,0,1,1,3,14,33,454,1,0,...,0,1,0,0,1,0,0,0,0,b
29,0,1,1,9,3,29,383,1,1,...,0,0,0,0,0,0,0,0,0,a
33,1,1,0,15,12,39,404,0,0,...,0,0,0,0,1,0,0,0,0,a
33,0,0,1,5,12,26,404,1,0,...,0,1,1,0,0,0,1,0,0,b
29,1,0,1,1,2,24,404,0,0,...,0,0,0,0,1,0,0,0,0,b
27,1,1,1,4,17,34,30,1,0,...,0,0,1,0,0,0,1,0,0,a
28,1,1,1,0,2,40,202,1,1,...,0,0,0,0,0,0,0,0,0,a
27,0,1,0,18,18,26,404,1,0,...,0,0,1,0,0,0,1,0,0,a


In [82]:
set.seed(1)
trainIndex = createDataPartition(train5$y, p = 0.7, list = FALSE)
traindata07 = train5[trainIndex, ]
traindata03 = train5[-trainIndex, ]

In [87]:
fitControl <- trainControl(method = "repeatedcv", number = 10, verboseIter = TRUE, classProbs = TRUE, repeats = 5)
tunegrid <- expand.grid(.mtry=c(3,5,10,15), .splitrule = "gini", .min.node.size = c(3,5,10,15))

In [ ]:
set.seed(300)
RF <- train(y~., data=traindata07, method="ranger", trControl = fitControl, tuneGrid = tunegrid)